In [ ]:
https://huggingface.co/docs/transformers/v4.18.0/en/main_classes/pipelines#transformers.TextClassificationPipeline

In [1]:
# install libraries
!pip install transformers

     |████████████████████████████████| 4.0 MB 12.9 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 57.8 MB/s 
     |████████████████████████████████| 895 kB 39.8 MB/s 
     |████████████████████████████████| 596 kB 63.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
!pip install simpletransformers

# Simple Classifiers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
df_training = pd.read_csv('/content/drive/MyDrive/LPJT/public_data.csv')
df_test = pd.read_csv('/content/drive/MyDrive/LPJT/input_data.csv')

In [4]:
df_training.head()

,id,message,label
0,8793,hi i want change my address from my credit card,updateaddress
1,3083,i need 4 fruit maple oatmeal 3 cold brew froze...,orderdrinkintent
2,5932,i wish to travel next month domestic airway,bookflight
3,12077,i need reimbursement my expenses,expensereport
4,6608,i need a copy of insurance for my car,getproofofinsurance


In [5]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
encoded_labels = labelEncoder.fit_transform(df_training['label'])

In [6]:
X = pd.concat([df_training['message'].rename('text'), pd.Series(encoded_labels, name='labels')], axis=1)
X.head()

,text,labels
0,hi i want change my address from my credit card,34
1,i need 4 fruit maple oatmeal 3 cold brew froze...,20
2,i wish to travel next month domestic airway,0
3,i need reimbursement my expenses,9
4,i need a copy of insurance for my car,13


In [12]:
n_labels = len(X.labels.value_counts())

37

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

In [15]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
model_args = ClassificationArgs(num_train_epochs=5)

model = ClassificationModel('roberta', 'roberta-base', use_cuda=True, num_labels=n_labels, args=model_args)
# from https://chanalytics-ai.medium.com/text-classification-with-simpletransformers-f8912372531b#:~:text=SimpleTransformers%20is%20a%20Natural%20Language,Generative%20Pre%2Dtrained%20Transformer).
# https://simpletransformers.ai/docs/classification-models/#specifying-the-number-of-classeslabels

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

https://simpletransformers.ai/docs/classification-data-formats/#multi-class-classification

https://simpletransformers.ai/docs/classification-models/#specifying-the-number-of-classeslabels

https://towardsdatascience.com/simple-transformers-introducing-the-easiest-bert-roberta-xlnet-and-xlm-library-58bf8c59b2a3

https://huggingface.co/docs/transformers/index

https://huggingface.co/docs/transformers/tasks/sequence_classification

In [16]:
model.train_model(X_train)

  0%|          | 0/9298 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1163 [00:00<?, ?it/s]

(1163, 0.6080057545151059)

In [17]:
from sklearn.metrics.cluster import adjusted_mutual_info_score
result, model_outputs, wrong_predictions = model.eval_model(X_val, ami=adjusted_mutual_info_score)

  0%|          | 0/2325 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/291 [00:00<?, ?it/s]

In [35]:
X_test = df_test['message'].rename('text')
predictions, raw_outputs = model.predict(X_test.values.tolist())

  0%|          | 0/2906 [00:00<?, ?it/s]

  0%|          | 0/364 [00:00<?, ?it/s]

In [38]:
transformed_predictions = labelEncoder.inverse_transform(predictions)

array(['reportbrokensoftware', 'orderpizzaintent', 'replacecard', ...,
       'orderburgerintent', 'getproofofinsurance', 'ordersaladintent'],
      dtype=object)

## Write predictions to zip for submission

In [39]:
my_submission = pd.DataFrame({'Id': df_test['id'], 'label': transformed_predictions})
print(my_submission.head())

      Id                 label
0  12123  reportbrokensoftware
1    244      orderpizzaintent
2   8221           replacecard
3  12856    startserviceintent
4  12108         expensereport


In [44]:
my_submission.to_csv('/content/drive/MyDrive/LPJT/roberta_coda_submit.zip', index=False)

**try out:**
- different bert models
- text cleaning steps
- number of epochs

meant for hybrid approach: google `using word embeddings for text classification`

Embeddings and flair example is probably start of hybrid approach


Hyperparaemter tuning tips for simpletransformer model?

Performance-wise: simpletransformer vs transformer

# Old stuff

In [10]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [13]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=2)

# Create a ClassificationModel
model = ClassificationModel("roberta", "distilroberta-base", args=model_args, use_cuda=False)
# model = ClassificationModel("roberta", "distilroberta-base", args=model_args, use_cuda=True)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

In [14]:
# Train the model
model.train_model(X_train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/9298 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1163 [00:00<?, ?it/s]

IndexError: ignored